# Few Shot Prompting PLLuM Evaluation

## Load the Data Source from DagsHub

In [ ]:
!pip install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 pandas==2.2.2 matplotlib==3.8.0 numpy==1.26.4
!pip install trl dagshub evaluate mlflow tqdm

  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached trl-0.12.1-py3-none-any.whl.metadata (10 kB)
  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached matplotlib-3.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtim

  Using cached dagshub-0.5.9-py3-none-any.whl.metadata (12 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached mlflow-2.22.0-py3-none-any.whl.metadata (30 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached gql-3.5.2-py2.py3-none-any.whl.metadata (9.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached treelib-1.7.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached pathvalidate-3.2.3-py3-none-any.whl.metadata (12 kB)
  Using cached boto3-1.38.15-py3-none-any.whl.metadata (6.6 kB)
  Using cached semver-3.0.4-py3-none-any.whl.metadata (6.8 kB)
  Using cached dagshub_annotation_converter-0.1.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached mlflow_skinny-2.22.0-py3-none-any.whl.metadata (31 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
 

In [ ]:
!pip install rouge_score bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from dagshub.data_engine import datasources

ds = datasources.get("informal2formal/mlflow", "synthetic_data_source")
dataset_uri = ds.head().dataframe["dagshub_download_url"].values[0]
df = pd.read_csv(dataset_uri)
df_train = df[df["split"] == "train"]
df_test = df[df["split"] == "test"]

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=185cd53c-7b61-40a2-b5e0-c6da73bb4f6f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c6fe28b7be71d0712e4dd82e89c81f92e78e810fb0405d219fd44c539f97774f




Accessing as sjerzykiewicz

Output()

In [ ]:
def gen_prompt(input_text, examples):
    prompt = "Twoim zadaniem jest formalizowanie wypowiedzi z zachowaniem jak największej liczby szczegółów i kontekstu. Nie skracaj zdań\n\nPrzykłady:\n"

    for informal, formal in examples:
        prompt += f"{informal}\n-> {formal}\n\n"

    prompt += f"{input_text}\n->"

    return prompt

In [ ]:
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer


bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
repo_id = 'CYFRAGOVPL/Llama-PLLuM-8B-instruct'
model = AutoModelForCausalLM.from_pretrained(repo_id,
                                             device_map="cuda:0",
                                             quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained("CYFRAGOVPL/Llama-PLLuM-8B-instruct")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

examples = df_train[["zdanie_nieformalne","zdanie_formalne"]].values.tolist()
test_dataset = df_test["zdanie_nieformalne"].apply(lambda x: gen_prompt(x, examples[:3])).tolist()
ver_dataset = df_test["zdanie_formalne"].tolist()

In [ ]:
from informal_to_formal.evaluation import Evaluator

def evaluate_language_model(
        pred: list[str], target: list[str]
    ) -> tuple[pd.DataFrame, dict]:
    evaluator = Evaluator(pd.DataFrame({
        "pred": pred,
        "target": target,
    }))
    evaluate_df_metrics, avg_metrics = evaluator.evaluate()
    return evaluate_df_metrics, avg_metrics

In [ ]:
import mlflow
import dagshub
import os


dagshub.init(repo_owner="informal2formal", repo_name="mlflow", mlflow=True)

mlflow.set_experiment(experiment_name="PLLuM Few-Shot-Prompting")

with mlflow.start_run(run_name="PLLuM FSP-1"):

    mlflow.log_param("base_model_name", "CYFRAGOVPL/Llama-PLLuM-8B-instruct")
    mlflow.log_param("prompt_template", gen_prompt("", examples[:3]))
    mlflow.log_param("dataset_uri", dataset_uri)

    # Test dataset evaluation using `Evaluator`
    model_predictions = [
        generator(input, max_length=1000, do_sample=False)
        for input in tqdm(test_dataset[:50])
    ]
    # get everything after last -> from predictions
    model_predictions_splitted = [pred[0]["generated_text"].split("->")[-1].strip() for pred in model_predictions]
    evaluate_df_metrics, avg_metrics = evaluate_language_model(
        model_predictions_splitted, ver_dataset[:50]
    )
    mlflow.log_metrics(avg_metrics)

Initialized MLflow to track repo "informal2formal/mlflow"

Repository informal2formal/mlflow initialized!

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Computing BERT scores: 100%|██████████| 50/50 [00:02<00:00, 20.84it/s]


🏃 View run PLLuM FSP-1 at: https://dagshub.com/informal2formal/mlflow.mlflow/#/experiments/8/runs/cf64501c9b12400ba71f209c47c8513c
🧪 View experiment at: https://dagshub.com/informal2formal/mlflow.mlflow/#/experiments/8
